## configure the env and load the dataset

In [ ]:
#!pip install huggingface-hub

In [ ]:
#!pip install datasets
import pandas

import torch



In [ ]:
from huggingface_hub import login
print("Hugging Face Hub is working")


from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

Hugging Face Hub is working


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

## check the structure and context of the dataset
1)the dataset type is datasetdict that holds multiple datasets.In this case,including train test unsupervised.every dataset include 2 column.one is text
the other is label .in label column 0 reprensent negative and 1 represent positive.

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


## split it by sentence

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

nltk.download('punkt_tab')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
train_ds = ds['train']

In [ ]:
def split_sentences(examples):
    # Tokenize each text into sentences
    sentences = [sent_tokenize(text) for text in examples['text']]

    # Flatten the nested lists and keep track of the label
    flat_sentences = []
    flat_labels = []
    for sentence_list, label in zip(sentences, examples['label']):
        flat_sentences.extend(sentence_list)
        flat_labels.extend([label] * len(sentence_list))  # Duplicate label for each sentence

    return {'sentence': flat_sentences, 'label': flat_labels}

# Apply the function to all splits in the DatasetDict
train_ds = train_ds.map(
    split_sentences,
    batched=True,  # Process in batches for efficiency
    remove_columns=['text'],  # Drop the original 'text' column
)


ValueError: Column to remove ['text'] not in the dataset. Current columns in the dataset: ['label', 'sentence']

In [ ]:
print(train_ds)

Dataset({
    features: ['label', 'sentence'],
    num_rows: 271057
})


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Save the dataset to Google Drive
train_ds.save_to_disk('/content/drive/My Drive/my_dataset')


NameError: name 'train_ds' is not defined

In [ ]:
from datasets import load_from_disk

In [ ]:
train_ds = load_from_disk('/content/drive/My Drive/my_dataset')

## Name column extraction NER

In [ ]:
import tensorflow as tf


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Num GPUs Available:  1


## Model_1
distilbert-NER


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/distilbert-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/distilbert-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer,batch_size=128)



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def extract_persons_for_columns(sentences):
    b_per_list = []
    i_per_list = []

    for sentence in sentences:
        ner_results = nlp(sentence)  # Apply NER to the sentence

        b_per = []
        i_per = []

        # Iterate through the results (list of dictionaries)
        for entity in ner_results:
            if entity['entity'] == 'B-PER' or entity['entity'] == 'I-PER':
                # Append the word, handling subword tokenization
                if entity['word'].startswith("##"):
                    # Append subword to the last token in the list
                    if entity['entity'] == 'B-PER' and b_per:
                        b_per[-1] += entity['word'][2:]  # Remove '##' and merge
                    elif entity['entity'] == 'I-PER' and i_per:
                        i_per[-1] += entity['word'][2:]
                else:
                    # Add a new word to the list
                    if entity['entity'] == 'B-PER':
                        b_per.append(entity['word'])
                    elif entity['entity'] == 'I-PER':
                        i_per.append(entity['word'])

        # Add the result to lists
        b_per_list.append(' '.join(b_per) if b_per else None)
        i_per_list.append(' '.join(i_per) if i_per else None)

    return {'b_per': b_per_list, 'i_per': i_per_list}

# Applying the function on the dataset
train_p = train_ds.map(
    lambda batch: extract_persons_for_columns(batch['sentence']),
    batched=True,  # Process multiple sentences in a batch
)


Map:   0%|          | 0/271057 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
train_p.save_to_disk('/content/drive/My Drive/my_dataset_2')

Saving the dataset (0/1 shards):   0%|          | 0/271057 [00:00<?, ? examples/s]

In [ ]:
print(train_p)

Dataset({
    features: ['label', 'sentence', 'b_per', 'i_per'],
    num_rows: 271057
})


In [ ]:
train_p = train_p.to_pandas()

# Check the resulting DataFrame
print(train_p.head())



   label                                           sentence b_per i_per
0      0  I rented I AM CURIOUS-YELLOW from my video sto...  None  None
1      0  I also heard that at first it was seized by U....  Lena  None
2      0  In particular she wants to focus her attention...  None  None
3      0  In between asking politicians and ordinary den...  None  None
4      0  Really, the sex and nudity scenes are few and ...  None  None


In [ ]:

train_p = train_p.dropna(subset=['b_per', 'i_per'], how='all')

In [ ]:
def pair_and_expand(df):
    expanded_rows = []
    for _, row in df.iterrows():
        # Handle None values by converting them to empty lists
        b_per_list = row['b_per'] if row['b_per'] is not None else []
        i_per_list = row['i_per'] if row['i_per'] is not None else []

        # Align the lists to avoid Cartesian product duplication
        max_len = max(len(b_per_list), len(i_per_list))

        # Extend lists to match lengths, using empty strings as placeholders
        b_per_list = b_per_list + [''] * (max_len - len(b_per_list))
        i_per_list = i_per_list + [''] * (max_len - len(i_per_list))

        # Append aligned rows to the expanded list
        for b, i in zip(b_per_list, i_per_list):
            expanded_rows.append({
                'sentence': row['sentence'],
                'b_per': b,
                'i_per': i,
                'label': row['label']
            })

    # Create a new DataFrame from the expanded rows
    return pd.DataFrame(expanded_rows)

# Convert the lists back into aligned rows
# Ensure b_per and i_per are strings and handle nulls
train_p['b_per'] = train_p['b_per'].fillna('').astype(str).str.split()
train_p['i_per'] = train_p['i_per'].fillna('').astype(str).str.split()



 #Pair and expand rows
train_p = pair_and_expand(train_p)


NameError: name 'train_p' is not defined

In [ ]:
train_p.head()

,sentence,b_per,i_per,label
0,I also heard that at first it was seized by U....,Lena,,0
1,"Even Ingmar Bergman, arguably their answer to ...",Ingmar,Bergman,0
2,"Even Ingmar Bergman, arguably their answer to ...",John,Ford,0
3,And those pretentious indie movies like The Br...,Vincent,Gallo,0
4,And those pretentious indie movies like The Br...,Chloe,Sevigny,0


In [ ]:
train_p.describe()

,label
count,160908.000000
mean,0.600324
std,0.489833
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [ ]:
train_p.to_csv('data_p.csv')

In [ ]:
from google.colab import files
files.download('data_p.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## model_2
spacy models

In [ ]:
import spacy
from spacy.lang.en.examples import sentences



In [ ]:
from tqdm import tqdm  # Import tqdm for progress bar

# Load spaCy model with GPU preference (if GPU is available)
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")



# List to store person names
person_names = []

# Get the total number of sentences
total_sentences = len(train_ds)

# Use tqdm to track progress over sentences
for doc in tqdm(nlp.pipe(train_ds["sentence"], batch_size=128), total=total_sentences, desc="Processing Sentences"):
    # Extract named entities of type 'PERSON'
    names = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    person_names.append(names)



Processing Sentences: 100%|██████████| 271057/271057 [07:59<00:00, 565.38it/s]

First sentence names: []


In [ ]:
train = train_ds.to_pandas()


In [ ]:
train.head(20)

,label,sentence,name
1,0,I also heard that at first it was seized by U....,[Lena]
6,0,"Even Ingmar Bergman, arguably their answer to ...","[Ingmar Bergman, John Ford]"
16,0,And those pretentious indie movies like The Br...,"[Vincent Gallo's, johnson, Chloe Sevigny]"
23,0,This film was probably inspired by Godard's Ma...,"[Godard, Masculin]"
25,0,Lena Nyman has to be most annoying actress in ...,[Lena Nyman]
27,0,"Comparing to Godard's film, intellectuality ha...",[Godard]
30,0,"Oh, brother...after hearing about this ridicul...",[Peggy Lee]
45,0,Whoever wrote the screenplay for this movie ob...,[Lucille Ball]
46,0,"I've never seen so many mistakes in a biopic, ...","[Jamestown, Desi]"
48,0,"In all, I believe that Lucille Ball is one of ...",[Lucille Ball]


In [ ]:
train["name"]=person_names

In [ ]:
# Filter out rows where the 'name' column contains an empty list
train = train[train["name"].apply(lambda x: len(x) > 0)]

# Display the resulting DataFrame
print(train)


        label                                           sentence  \
1           0  I also heard that at first it was seized by U....   
6           0  Even Ingmar Bergman, arguably their answer to ...   
16          0  And those pretentious indie movies like The Br...   
23          0  This film was probably inspired by Godard's Ma...   
25          0  Lena Nyman has to be most annoying actress in ...   
...       ...                                                ...   
271050      1  And made tonnes more money.<br /><br />The son...   
271051      1  You have a love a lyric that includes the line...   
271054      1  The story centers around Barry McKenzie who mu...   
271055      1  Being about the grossest Aussie shearer ever t...   
271056      1  The songs of Barry McKenzie(Barry Crocker) are...   

                                             name  
1                                          [Lena]  
6                     [Ingmar Bergman, John Ford]  
16      [Vincent Gallo's, j

In [ ]:
def split_and_duplicate(df):
    expanded_rows = []
    for _, row in df.iterrows():

        for name in row['name']:
            expanded_rows.append({
                'label': row['label'],
                'sentence': row['sentence'],
                'name': name
            })  # Remove square brackets and split by commas


    return pd.DataFrame(expanded_rows)

# Apply the function to split and duplicate
train_s = split_and_duplicate(train)





In [ ]:
train_s.head(20)

,label,sentence,name
0,0,I also heard that at first it was seized by U....,Lena
1,0,"Even Ingmar Bergman, arguably their answer to ...",Ingmar Bergman
2,0,"Even Ingmar Bergman, arguably their answer to ...",John Ford
3,0,And those pretentious indie movies like The Br...,Vincent Gallo's
4,0,And those pretentious indie movies like The Br...,johnson
5,0,And those pretentious indie movies like The Br...,Chloe Sevigny
6,0,This film was probably inspired by Godard's Ma...,Godard
7,0,This film was probably inspired by Godard's Ma...,Masculin
8,0,Lena Nyman has to be most annoying actress in ...,Lena Nyman
9,0,"Comparing to Godard's film, intellectuality ha...",Godard


In [ ]:
train_s.to_csv("train_s.csv")

In [ ]:
train_s.isnull().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118481 entries, 0 to 118480
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   label     118481 non-null  bool 
 1   sentence  118481 non-null  bool 
 2   name      118481 non-null  bool 
dtypes: bool(3)
memory usage: 347.2 KB


In [ ]:
from google.colab import files
files.download('train_s.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>